In [1]:
import os
from datetime import datetime
# Make logging folder named according to current time
start_time_str = datetime.today().strftime('%d_%m_%Y__%H_%M_%S')
log_dir = './logs/'f"log_{start_time_str}"
os.makedirs(log_dir)

# Initialize logger that adds to text file in current logging folder
from logging_utils import *

init_logger(save_dir=log_dir)
logger = get_logger()




In [2]:
import pathlib
import subprocess
# Get the absolute path to your repository, 
# no matter where you are running this code from

repo_path = os.getcwd() 


git_branch = subprocess.check_output(
    ["git", "-C", repo_path, "rev-parse", "--abbrev-ref", "HEAD"]).strip().decode('UTF-8')

git_commit_short_hash = subprocess.check_output(
    ["git", "-C", repo_path, "describe", "--always"]).strip().decode('UTF-8')


In [3]:
import numpy as np

dim=100
x0=np.squeeze(np.zeros((dim,1)))
x0Type='Zero Vector'

FD_StencilMat=np.zeros((dim,dim))
b = np.ones(dim-1)
c = -2*np.ones(dim)
np.fill_diagonal(FD_StencilMat[1:], b)
np.fill_diagonal(FD_StencilMat[:,1:], b)
np.fill_diagonal(FD_StencilMat, c)
A=FD_StencilMat
AType='1D Laplacian'


nmax_iter = dim
retrain_freq=10
e1 = 1e-8
e2 = 1e-10
nmax_iter = 50
restart   = 64

n_steps =400
InputDim=dim
HiddenDim=2*dim
HiddenDim2=1*dim
OutputDim=dim

In [4]:
# Initial log message
logger.info("Benchmark 100 dim problem")

2020-05-13 00:25:17 [INFO] <ipython-input-4-ac6c0d00fb29> 2 - Benchmark 100 dim problem


In [5]:
# Set up parameter class
class MyParamsClass():
    def __init__(self, AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,git_branch,git_commit_short_hash):
        self.AType = AType
        self.dim = dim
        self.nmax_iter = nmax_iter
        self.restart = restart
        self.n_steps =n_steps
        self.retrain_freq=retrain_freq
        self.e1=e1
        self.e2=e2
        self.x0Type=x0Type
        self.InputDim=InputDim
        self.HiddenDim=HiddenDim
        self.HiddenDim2=HiddenDim2
        self.OutputDim=OutputDim
        self.git_branch=git_branch
        self.git_commit_short_hash=git_commit_short_hash

In [6]:
import os
import json

params = MyParamsClass(AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,git_branch,git_commit_short_hash)
sorted_params_dict = {k: params.__dict__[k] for k 
                      in sorted(params.__dict__.keys())}

params_filepath = log_dir+'/params.json'
json.dump(sorted_params_dict, open(params_filepath, 'w'), indent=4)

In [7]:
from src_dir import *

In [8]:
# @nn_preconditioner(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
# def MLGMRES(A, b, x0, e, nmax_iter,IterErr0_sum,IterErr0,ProbCount,Add,restart,debug):
#     return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [9]:
@timer
@nn_preconditioner_timed(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
def MLGMRES(A, b, x0, e, nmax_iter,IterErr0_sum,IterErr0,ProbCount,Add,restart,debug,refine):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)
    

In [10]:
@timer
def GMRES_timed(A, b, x0, e, nmax_iter,restart,debug):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [11]:
IterErr0_Avg=0.0 
IterErr0=0

debug=True
Add1=False
Add2=True
refine1=False
refine2=True

run_time=0.0
run_time_ML=0.0
trainTime_total=0.0
forwardTime_Total=0.0
run_time_ML_NoAdd=0.0


for ProbIdx in range(n_steps):
    b =6*np.squeeze(np.random.randn(1,dim))  #random b
    b_norm=np.linalg.norm(b)
    

    OutList1,run_time1_ML_NoAdd=MLGMRES(A, b/b_norm, x0, e1, nmax_iter,IterErr0_Avg,IterErr0,ProbIdx,Add1,restart,debug,refine1)  # ML no learning 
    Out=OutList1[0]
    OutList2,run_time2_ML_NoAdd=MLGMRES(A, b/b_norm, Out[-1], e2, nmax_iter,IterErr0_Avg,IterErr0,ProbIdx,Add1,restart,debug,refine2)
    run_time_ML_NoAdd=run_time1_ML_NoAdd+run_time2_ML_NoAdd+run_time_ML_NoAdd





    OutList1,run_time1_ML=MLGMRES(A, b/b_norm, x0, e1, nmax_iter,IterErr0_Avg,IterErr0,ProbIdx,Add1,restart,debug,refine1)  # normalize b for optimal NN performance.
    Out=OutList1[0]
    IterErr0_Avg=OutList1[1]
    IterErr0=OutList1[2]
    forwardTime1=OutList1[4]

    OutList2,run_time2_ML=MLGMRES(A, b/b_norm, Out[-1], e2, nmax_iter,IterErr0_Avg,IterErr0,ProbIdx,Add2,restart,debug,refine2)
    Out2=OutList2[0]
    IterErr0_Avg=OutList2[1]
    IterErr0=OutList2[2]
    trainTime=OutList2[3]
    forwardTime2=OutList2[4]

    forwardTime_Total=forwardTime1+forwardTime2+forwardTime_Total
    trainTime_total=trainTime+trainTime_total
    run_time_ML=run_time1_ML+run_time2_ML+run_time_ML






    NonML_Out1,run_time1=GMRES_timed(A, b/b_norm, x0, e1, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    NonML_Out2,run_time2=GMRES_timed(A, b/b_norm, NonML_Out1[-1], e2, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    run_time=run_time1+run_time2+run_time

    print(ProbIdx)


logger.info("Runtime of Non-decorated version is:")
logger.info(run_time)

logger.info("Runtime of MLGMRES decorator is:")
logger.info(run_time_ML)


logger.info("Runtime of training (backprop) is:")
logger.info(trainTime_total)

logger.info("Runtime of forward function is:")
logger.info(forwardTime_Total)

logger.info("Runtime of ML GMRES decorator with NO learning is:")
logger.info(run_time_ML_NoAdd)


919262265 0.8724589884882394
237
0.8630915226835916 0.8724196293882199
0.8630915226835916 0.8724196293882199
0.8630915226835916 0.8724196293882199
238
0.891895309765745 0.8725011175906363
0.891895309765745 0.8725011175906363
0.891895309765745 0.8725011175906363
239
0.7815951565712356 0.8721223427530554
0.7815951565712356 0.8721223427530554
0.7815951565712356 0.8721223427530554
240
0.7686135726563282 0.8716928457816998
0.7686135726563282 0.8716928457816998
0.7686135726563282 0.8716928457816998
241
0.7918446169084412 0.8713628944227193
0.7918446169084412 0.8713628944227193
0.7918446169084412 0.8713628944227193
242
0.7921759967962398 0.871037022416026
0.7921759967962398 0.871037022416026
0.7921759967962398 0.871037022416026
243
0.7629284070961123 0.8705939543204526
0.7629284070961123 0.8705939543204526
0.7629284070961123 0.8705939543204526
244
0.9213210690213138 0.8708010037682112
0.9213210690213138 0.8708010037682112
0.9213210690213138 0.8708010037682112
245
0.9670663042050545 0.87119232

2020-05-12 18:34:58 [INFO] <ipython-input-12-8d65b0681b78> 1 - Runtime of training (backprop) is:
2020-05-12 18:34:58 [INFO] <ipython-input-12-8d65b0681b78> 2 - 6.218440221025958
